In [110]:
from pprint import pprint
from sklearn import metrics
import numpy as np
import sys

def _metrics_report_to_df(ytrue, ypred):
    precision, recall, fscore, support = metrics.precision_recall_fscore_support(ytrue, ypred)
    acc = metrics.accuracy_score(ytrue, ypred)
    classification_report = pd.concat(map(pd.DataFrame, [[acc,acc], fscore, precision, recall, support]), axis=1)
    classification_report.columns = ["accuracy", "f1-score", "precision", "recall", "support"]
    return(classification_report)

import pandas as pd
# data="/meladyfs/newyork/krsharma/kdd_data/kwon/sentiments.txt"
data="/meladyfs/newyork/krsharma/kdd_data/twitter-ma/twitter-ma.csv"

df = pd.read_csv(data)
print df.head()
print df.columns
# sys.exit()

senti = df.groupby(['fact'], as_index=False).mean()
label = (senti['sentiment_score']>0)*1

print senti.head()
print senti.columns
print len(label)

ground = []
for row in senti['fact']:
    if row.split("_")[0] == 'N':
        ground.append(0)
    else:
        ground.append(1)
print len(ground)
        
report = _metrics_report_to_df(ground, label) # metrics.classification_report(labels, assigned, target_names=self.keys, output_dict=True)
pprint(report)       

# Kwon

# accuracy  f1-score  precision    recall  support
# 0  0.531532  0.527273   0.491525  0.568627       51
# 1  0.531532  0.535714   0.576923  0.500000       60

# MA
# accuracy  f1-score  precision    recall  support
# 0  0.486218  0.572477   0.474886  0.720554      433
# 1  0.486218  0.356354   0.516000  0.272152      474


      website    fact  object  sentiment_score
0  1003856029  N_E312  N_E312          -0.6597
1  1013430282  N_E312  N_E312          -0.7149
2  1014445069  N_E312  N_E312          -0.7149
3   101596871  N_E312  N_E312           0.5267
4    10228272  N_E312  N_E312          -0.7412
Index([u'website', u'fact', u'object', u'sentiment_score'], dtype='object')
                       fact       website  sentiment_score
0               N_Airfrance  2.313271e+07        -0.313637
1                N_Airliner  1.396921e+07         0.129432
2                  N_Amanda  2.438650e+07        -0.103014
3                 N_AnnieLe  4.059135e+07        -0.194677
4  N_BarnesNobleObamaMonkey  1.760870e+07         0.238274
Index([u'fact', u'website', u'sentiment_score'], dtype='object')
992
992
   accuracy  f1-score  precision    recall  support
0  0.469758  0.532028   0.474603  0.605263      494
1  0.469758  0.388372   0.461326  0.335341      498


In [103]:
assigned = np.array(np.loadtxt("untitled1.txt"))
print assigned[0:5]
ground = np.loadtxt("/meladyfs/newyork/krsharma/kdd_data/twitter-ma/train_labels.txt")
print ground[0:5]
report = _metrics_report_to_df(ground, assigned) # metrics.classification_report(labels, assigned, target_names=self.keys, output_dict=True)
pprint(report)

[1. 0. 1. 0. 1.]
[0. 1. 0. 1. 1.]
   accuracy  f1-score  precision    recall  support
0  0.365927  0.365288   0.364185  0.366397      494
1  0.365927  0.366566   0.367677  0.365462      498


In [3]:
# Number of posts in the cascade
# Time: Total time length of cascade, Avg time gap between posts in cascade
## ignore tree: Avg Depth of tree, Avg Breadth of tree 
 
# data="/meladyfs/newyork/krsharma/kdd_data/twitter-ma/train_cascades.txt"
# ground_data="/meladyfs/newyork/krsharma/kdd_data/twitter-ma/train_labels.txt"
data="/home/krsharma/kdd_netinference/kwon/train_cascades.txt"
ground_data="/home/krsharma/kdd_netinference/kwon/train_labels.txt"
import operator, sys
from sklearn import preprocessing

def extract_feats(ulist, tlist, top_users_set):
    total_time = tlist[-1]
    num_posts = len(ulist)
    time_gap = tlist[1:] - tlist[0:-1]
    frac_top_users = len(set(ulist) & top_users_set)*1.0/len(ulist)
    return [num_posts, total_time, np.mean(time_gap), frac_top_users]
    
    # feats = extract_feats(np.array([1,2,3]), np.array([2,5,6]))


cascades = []
upart = {}

tot_eng = 0
for line in open(data):
    activations = line.split(" ")
    tot_eng += len(activations)
    ulist, tlist = [], []
    for a in activations:
        u, t = a.strip().split("-")
        if u not in upart:
            upart[u] = 1
        else: upart[u] += 1
        ulist.append(u); tlist.append(float(t))
    cascades.append((ulist, tlist))
    # break
print "user", len(upart), tot_eng
sys.exit()
    
sorted_x = sorted(upart.items(), key=operator.itemgetter(1), reverse=True)
top_users = sorted_x[:5000]
top_users_set = set()
for u, count in top_users:
    top_users_set.add(u)

X = []
for ulist, tlist in cascades:
    cas_features = extract_feats(np.array(ulist), np.array(tlist), top_users_set)
    X.append(cas_features)

X = np.array(X)
print X.shape
score = np.mean(X, axis=0)
score[1] = score[1]/3600.0; score[2] = score[2]/3600.0 
print list(score) # [num_posts, total_time, np.mean(time_gap), frac_top_users]
# sys.exit()
# print X

from sklearn.cluster import KMeans
import numpy as np

# kmeans = KMeans(n_clusters=2, random_state=None).fit(preprocessing.normalize(X))
kmeans = KMeans(n_clusters=2, random_state=None).fit(X)
assigned = kmeans.labels_# array([0, 0, 0, 1, 1, 1], dtype=int32)
# print assigned
print len(assigned)

ground = np.loadtxt(ground_data)
report = _metrics_report_to_df(ground, assigned) # metrics.classification_report(labels, assigned, target_names=self.keys, output_dict=True)
pprint(report)
report = _metrics_report_to_df(ground, 1-assigned) # metrics.classification_report(labels, assigned, target_names=self.keys, output_dict=True)
pprint(report)
# print kmeans.predict([[0, 0], [4, 4]]) # array([0, 1], dtype=int32)
# print kmeans.cluster_centers_ # array([[1., 2.], [4., 2.]])

user 117824 192350


SystemExit: 

In [123]:
kwon (unnormalized)
accuracy  f1-score  precision    recall  support
0  0.405405  0.297872   0.325581  0.274510       51
1  0.405405  0.484375   0.455882  0.516667       60

   accuracy  f1-score  precision    recall  support
0  0.594595  0.621849   0.544118  0.725490       51
1  0.594595  0.563107   0.674419  0.483333       60

t-ma (normalized)
(992, 4)
[597.1683467741935, 1982.9470718525986, 64.61494570103345, 0.16557653119819682]
992
   accuracy  f1-score  precision    recall  support
0  0.490927  0.595677   0.492715  0.753036      494
1  0.490927  0.312925   0.485232  0.230924      498
  accuracy  f1-score  precision    recall  support
0  0.509073  0.333789   0.514768  0.246964      494
1  0.509073  0.611333   0.507285  0.769076      498

SyntaxError: invalid syntax (<ipython-input-123-2f71aaa713e4>, line 2)

In [79]:
s= [13.36, 0.187, 1.49, 0.589, 0.269]
    
print np.mean(s), np.std(s)
print np.median(s)

3.179 5.111433184538363
0.589
